# Examples of Simodes use

V textu dále jsou pomocí tří příkladů demonstrovány základní funkcionality knihovny simodes, která poskytuje nástroje pro propojení simulací založených na modelech definovaných diferenciálními rovnicemi a simulací založených na událostních modelech.

V poslední kapitole jsou uvedeny některé implementační detaily u vybraných funkcí.

In [1]:
!pip install simodes

## Příklad A

V tomto příkladě je ukázáno, jakým způsobem lze v simulaci pracovat s modelem definovaným pomocí diferenciální rovnice.

### Importy z knihovny

In [2]:
import simodes
from simodes import Simulator
from simodes import simpleODESolver
from simodes import createDataSelector
#print(dir(simodes))

### Inicializace simulace

Simulační prostředí je závislé na třídě Simulator. Vytvořením její instance dostáváme k dispozici veškeré metody, které jsou nezbytné pro událostní (DES) simulaci a simulaci založené na diferenciálních rovnicích (ODE).

In [3]:
sim = Simulator()
currentState = sim.GetState()
print(currentState)

{'odeModels': {}, 'eventList': {'events': [], 'activeEvent': None}, 'logs': []}


### Definice modelu

V případě, kdy je se jedná o simulaci založenou na modelech popsaných diferenciálními rovnicemi, je nutné tyto modely definovat. Model je vyjádřen funkcí se dvěma parametry, přičemž první parametr určuje čas a druhý stav modelu. Funkce navrací hodnotu, která je derivací stavu v daném čase. Níže uvedený model popisuje pohyb hmotného bodu ve 2D prostoru, přičemž v ose y působí gravitace se zrychlením g=9.81.

In [4]:
def model2D(time, state):
    return [state[2], state[3], 0, -9.81]

### Zanesení modelu do simulace

Při vkládání modelu do simulace je nutné definovat:

- diferenciální rovnici (model2D), 

- čas, kdy je simulace pohybu zahájena (0s),

- počáteční stav ([0, 0, 10, 10]),

- mezní čas, tedy čas, kdy výpočty končí (1e300) a

- maximální krok (0.0625)

V simulaci lze současně zpracovávat více ODE modelů, v tomto případě jsou to dva se stejnou diferenciální rovnicí.
Simulátor každému vloženému modelu přiřadí unikátní identifikátor. Prostřednictvím tohoto identifikátoru lze získávat informace o stavu modelu.

In [5]:
solverA = simpleODESolver(
    model2D, 0, state0=[0, 0, 10, 10], t_bound=1e300, max_step=0.0625)
modelIdA = sim.AttachODESolver(solverA)

solverB = simpleODESolver(
    model2D, 0, state0=[0, 0, 10, 10], t_bound=1e300, max_step=0.0625)
modelIdB = sim.AttachODESolver(solverB)

Po vložení modelu lze zjistit stav. Data o simulaci lze získat voláním funkce ```sim.GetState()```.

Pokud u modelu je atribut ```destroyed``` nastaven na hodnotu ```true```, simulace modelu je ukončena (model se již dále nebude pohybovat).

In [6]:
currentState = sim.GetState()
print(currentState)

{'odeModels': {'kkvpwgotvt': {'destroyed': False, 'state': {'time': 0, 'y': [0, 0, 10, 10], 'yd': [10, 10, 0, -9.81]}}, 'ojkozcquko': {'destroyed': False, 'state': {'time': 0, 'y': [0, 0, 10, 10], 'yd': [10, 10, 0, -9.81]}}}, 'eventList': {'events': [{'time': 0, 'executor': <function Simulator.AttachODESolver.<locals>.stepper at 0x7f140f6b2a60>, 'kwargs': {'state': {'time': 0, 'y': [0, 0, 10, 10], 'yd': [10, 10, 0, -9.81]}}}, {'time': 0, 'executor': <function Simulator.AttachODESolver.<locals>.stepper at 0x7f140f6b2ca0>, 'kwargs': {'state': {'time': 0, 'y': [0, 0, 10, 10], 'yd': [10, 10, 0, -9.81]}}}], 'activeEvent': None}, 'logs': []}


### Příprava metod pro transformaci dat

Identifikátory získané při zavedení modelu do simulace slouží pro extraci dat ze simulace. Jako podpůrný prvek lze využít funkci createDataSelector.

Proměnná ```masterMap``` definovaná níže definuje, které modely jsou v simulaci sledovány. Názvy ```bulletA_``` a ```bulletB_``` se použijí později.

In [7]:
masterMap = {
    'bulletA_': lambda item: item[modelIdA],
    'bulletB_': lambda item: item[modelIdB],
}

Proměnná ```dataDescriptor``` slouží pro extrakci vybraných parametrů entit. V tomto případě se jedná o čas a první dva prvky stavu modelu, které v našem případě představují souřadnice x a y.

In [8]:
dataDescriptor = {
    't': lambda item: item['state']['time'],
    'x': lambda item: item['state']['y'][0],
    'y': lambda item: item['state']['y'][1]
}

Na základě definovaných proměnných ```masterMap``` a ```dataDescriptor``` je vytvořena funkce  ```dataSelector```, u které je předpoklad, že bude využita v průběhu simulace.

In [9]:
dataSelector = createDataSelector(masterMap, dataDescriptor)

Demonstrace využití ```dataSelector```, kdy z celých dat je vybrána je požadovaná část.

In [10]:
simData = sim.GetState()
selectedData = dataSelector(simData['odeModels'])
print(selectedData)

{'bulletA_t': 0, 'bulletA_x': 0, 'bulletA_y': 0, 'bulletB_t': 0, 'bulletB_x': 0, 'bulletB_y': 0}


### Cyklus simulace

Před spuštěním cyklu simulace je inicializována proměnná ```results```, do které budou postupně vkládány výsledky simulace. 

> Pozor, níže použitý cyklus simulace může být nekonečný, je tedy potřeba definovat podmínku ukončení. V tomto konkrétním případě je simulace ukončena po 6 krocích. 

V každém kroku je z celkových dat simulace vybrána zájmová podmnožina pomocí funkce ```dataSelector``` a její výstup je vložen do pole results.
Cyklus simulace, jak je definován níže, obsahuje v proměnné ```index``` číslo kroku a ```currentResult``` informace o stavu všech modelů v simulaci.

In [11]:
results = []
for index, currentResult in enumerate(sim.Run()):
    partialResult = dataSelector(currentResult)
    results.append(partialResult)
    if index >= 5:
        break

Po ukončení cyklu simulace je možné vypsat souhrnné výsledky.

In [12]:
print(results)

[{'bulletA_t': 0, 'bulletA_x': 0, 'bulletA_y': 0, 'bulletB_t': 0, 'bulletB_x': 0, 'bulletB_y': 0}, {'bulletA_t': 0, 'bulletA_x': 0, 'bulletA_y': 0, 'bulletB_t': 0, 'bulletB_x': 0, 'bulletB_y': 0}, {'bulletA_t': 9.999000075938193e-05, 'bulletA_x': 0.0009999000075938194, 'bulletA_y': 0.0009998509674025836, 'bulletB_t': 0, 'bulletB_x': 0, 'bulletB_y': 0}, {'bulletA_t': 9.999000075938193e-05, 'bulletA_x': 0.0009999000075938194, 'bulletA_y': 0.0009998509674025836, 'bulletB_t': 9.999000075938193e-05, 'bulletB_x': 0.0009999000075938194, 'bulletB_y': 0.0009998509674025836}, {'bulletA_t': 0.0010998900083532014, 'bulletA_x': 0.010998900083532014, 'bulletA_y': 0.01099296622039253, 'bulletB_t': 9.999000075938193e-05, 'bulletB_x': 0.0009999000075938194, 'bulletB_y': 0.0009998509674025836}, {'bulletA_t': 0.0010998900083532014, 'bulletA_x': 0.010998900083532014, 'bulletA_y': 0.01099296622039253, 'bulletB_t': 0.0010998900083532014, 'bulletB_x': 0.010998900083532014, 'bulletB_y': 0.01099296622039253}]


Výsledky lze zpracovat dalšími standardními postupy, například zobrazit jako tabulku.

In [13]:
import pandas as pd

def displayData(data):
    df = pd.DataFrame(data)
    display(df)

In [14]:
displayData(results)

,bulletA_t,bulletA_x,bulletA_y,bulletB_t,bulletB_x,bulletB_y
0,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000
1,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000
2,0.0001,0.001000,0.001000,0.0000,0.000000,0.000000
3,0.0001,0.001000,0.001000,0.0001,0.001000,0.001000
4,0.0011,0.010999,0.010993,0.0001,0.001000,0.001000
5,0.0011,0.010999,0.010993,0.0011,0.010999,0.010993


Protože se jedná o framework podporující událostní simulaci i simulaci založenou na ODE modelech, je zcela běžné, že se v tabulce objevují stejné časy. Současně je patřičné uvést, že v simulaci se "pohybuje vždy jedním objektem" a tedy časy objektů nemusí být synchronní.

## Příklad B

V tomto příkladu je ukázáno, jak lze v simulaci pracovat s událostmi.

### Importy z knihovny

In [15]:
import simodes
from simodes import Simulator
from simodes import simpleODESolver
from simodes import createDataSelector
#print(dir(simodes))

### Inicializace simulace

Simulační prostředí je závislé na třídě Simulator. Vytvořením její instance dostáváme k dispozici veškeré metody, které jsou nezbytné pro událostní (DES) simulaci a simulaci založené na diferenciálních rovnicích (ODE).

In [16]:
sim = Simulator()
currentState = sim.GetState()
print(currentState)

{'odeModels': {}, 'eventList': {'events': [], 'activeEvent': None}, 'logs': []}


### Definice událostí v simulaci

V případě, kdy je se jedná o simulaci založenou na událostech, je nutné tyto události definovat. Událost je funkcí, jejímž prvním parametrem je čas. 

In [17]:
def eventComeIn(time):
    print(f'At {time}s an event occurs')

### Naplánování událost v simulaci

Prvním parametrem je čas, kdy k události dojde a druhým parametrem, které funkce bude v daném čase simulace vyvolána.

In [18]:
sim.AddEvent(0, eventComeIn)

### Definice událostí v simulaci II

Častějším typem události, než jaká byla demonstrována výše je událost, na kterou navazuje další událost. Toto lze řešit naplánování další události v rámci obsluhy aktuální události. Událost může mít více parametrů.

> V simulaci lze samozřejmě definovat více typů událostí s různou obsluhou

V příkladu je popsán systém hromadné obsluhy s frontou FIFO a jednou obslužnou linkou.

In [19]:
import random

queue = []
def eventComeInEx(time, addEvent):
    print(f'At {time}s someone comes in')
    queue.append(time)
    nextTime = time + random.uniform(1.5, 3)
    addEvent(nextTime, eventComeInEx, addEvent=addEvent)
    addEvent(time, tryBeginService, addEvent=addEvent)
    
service = {'who': None}
def tryBeginService(time, addEvent):
    if len(queue) > 0: # queue is not empty
        if service['who'] is None: # service is ready
            timeIn = queue.pop()
            timeOut = timeIn + random.uniform(0.5, 2.5)
            service['who'] = {'systemIn': timeIn, 'serviceBegin': time, 'systemOut': timeOut}
            addEvent(timeOut, eventServiceEnd, addEvent=addEvent)
            
def eventServiceEnd(time, addEvent):
    item = service['who']
    print(f'At {time}s {item} leaves the system')
    service['who'] = None
    addEvent(time, tryBeginService, addEvent=addEvent)
    

### Naplánování události v simulaci

V tomto případě událost je naplánována s extra parametrem ```addEvent```. Tento parametr je předán funkci, která událost obslouží (```eventComeInEx```).

In [20]:
sim.AddEvent(0, eventComeInEx, addEvent=sim.AddEvent)

### Cyklus simulace

> Pozor, níže použitý cyklus simulace může být nekonečný, je tedy potřeba definovat podmínku ukončení. V tomto konkrétním případě je simulace ukončena nejpozději po 6 krocích. 

In [21]:
results = []
for index, currentResult in enumerate(sim.Run()):
    if index >= 5:
        break

At 0s an event occurs
At 0s someone comes in
At 1.5822921669943888s {'systemIn': 0, 'serviceBegin': 0, 'systemOut': 1.5822921669943888} leaves the system
At 2.2386463046990777s someone comes in


## Příklad C

V tomto příkladu je ukázáno jak v simulaci průběžně ukládat informace.

### Importy z knihovny

In [22]:
import simodes
from simodes import Simulator
from simodes import simpleODESolver
from simodes import createDataSelector
#print(dir(simodes))

### Inicializace simulace

Simulační prostředí je závislé na třídě Simulator. Vytvořením její instance dostáváme k dispozici veškeré metody, které jsou nezbytné pro událostní (DES) simulaci a simulaci založené na diferenciálních rovnicích (ODE).

In [23]:
sim = Simulator()
currentState = sim.GetState()
print(currentState)

{'odeModels': {}, 'eventList': {'events': [], 'activeEvent': None}, 'logs': []}


### Definice událostí v simulaci

Při obsluze události jsou ukládány informace do logu simulace.
> V simulaci lze ukládat různé typy logů. Ty jsou odlišeny ukládanými parametry.

In [24]:
import random

queue = []
def eventComeInEx(time, addEvent, addLog):
    addLog(f'At {time}s someone comes in', time=time)
    queue.append(time)
    nextTime = time + random.uniform(1.5, 3)
    addEvent(nextTime, eventComeInEx, addEvent=addEvent, addLog=addLog)
    addEvent(time, tryBeginService, addEvent=addEvent, addLog=addLog)
    
service = {'who': None}
def tryBeginService(time, addEvent, addLog):
    if len(queue) > 0: # queue is not empty
        if service['who'] is None: # service is ready
            timeIn = queue.pop()
            timeOut = timeIn + random.uniform(0.5, 2.5)
            service['who'] = {'systemIn': timeIn, 'serviceBegin': time, 'systemOut': timeOut}
            addEvent(timeOut, eventServiceEnd, addEvent=addEvent, addLog=addLog)
            
            
def eventServiceEnd(time, addEvent, addLog):
    item = service['who']
    
    addLog(f'At {time}s item leaves the system', time=time, item=item)
    service['who'] = None
    addEvent(time, tryBeginService, addEvent=addEvent, addLog=addLog)
    

### Naplánování události v simulaci

V tomto případě událost je naplánována s extra parametrem ```addEvent```. Tento parametr je předán funkci, která událost obslouží (```eventComeInEx```).

In [25]:
sim.AddEvent(0, eventComeInEx, addEvent=sim.AddEvent, addLog=sim.AddLog)

### Cyklus simulace

> Pozor, níže použitý cyklus simulace může být nekonečný, je tedy potřeba definovat podmínku ukončení. V tomto konkrétním případě je simulace ukončena nejpozději po 6 krocích. 

In [26]:
results = []
for index, currentResult in enumerate(sim.Run()):
    if index >= 5:
        break

### Výpis událostí

Během simulace i po jejím ukončení lze provést výpis zaznamenaných událostí

In [27]:
state = sim.GetState()
logs = state['logs']
for item in logs:
    print(item)

{'msg': 'At 0s someone comes in', 'time': 0}
{'msg': 'At 1.809040473507442s item leaves the system', 'time': 1.809040473507442, 'item': {'systemIn': 0, 'serviceBegin': 0, 'systemOut': 1.809040473507442}}
{'msg': 'At 2.6763418177499236s someone comes in', 'time': 2.6763418177499236}


## Imports and Special Functions

### createDataSelector Function

Funkce ```createDataSelector``` slouží pro přípravu jednoduché transformace dat v průběhu simulace. Umožňuje z celkové informace o simulaci extrahovat jen dílčí prvky. Její použití je uvedeno v další části.

> Implementaci není nutno studovat. Funkci je možné importovat přímo z knihovny.

In [28]:
def createDataSelector(masterMaps, maps):
    def extractor(dataItem):
        result = {}
        for masterName, masterFunc in masterMaps.items():
            row = masterFunc(dataItem)
            for name, func in maps.items():
                result[masterName + name] = func(row)
        return result
    return extractor

### simpleODESolver Function

Funkce ```simpleODESolver``` je funkcí, která na základě modelu, jeho počátečního stavu generuje v daném časové intervalu stavy modelu. Funkce je koncipována jako generátor a vrací jednotlivé stavy na vyžádání. Tato konkrétní implementace je založena na metodě RungeKutta. V případě potřeby lze implementaci změnit / zpřesnit. To ovšem obvykle, vzhledem ke způsobu práce simulačního prostředí s více modely, není třeba.

> Implementaci není třeba studovat. Funkci je možné importovat přímo z knihovny.

In [29]:
import scipy.integrate as integrate # for numerical solution od differential equations

def simpleODESolver(model, t0, state0, t_bound, max_step):
    if not callable(model):
        raise ValueError('Model must be callable')

    solver = integrate.RK45(fun = model, t0 = t0, y0 = state0, t_bound = t_bound, max_step = max_step)
    currentItem = {'time': solver.t, 'y': [*state0], 'yd': [*model(t0, state0)]}
    while True:
        yield currentItem # send signal, inform about current result
        message = solver.step()
        currentItem = {'time': solver.t, 'y': [*solver.y], 'yd': [*model(solver.t, solver.y)]}
        if (not(solver.status == 'running')):
            break